In [1]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.9 MB/s eta 0:00:00


In [2]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 불러오기

In [44]:
df = pd.read_csv("/content/drive/MyDrive/TM/Data/Labeled.csv", index_col = 0)
df.head()

,sentence,label,senti
0,Would be super sexy if you are only about 4 ft...,1,1
1,"I'm 5'5"" and this barely stretched to the unde...",1,2
2,Don't waste your $,3,2
3,Tiny fit and terrible print!,1,2
4,Got it for my brother who normally wears a large.,1,2


In [48]:
df.groupby('senti').count()

,sentence,label
senti,,
0,3275,3275
1,122532,122532
2,120204,120204


In [49]:
df = df[df['senti'] != 0]

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242736 entries, 0 to 246010
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sentence  242736 non-null  object
 1   label     242736 non-null  int64 
 2   senti     242736 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.4+ MB


In [56]:
df0_1 = df.loc[(df['label'] == 0) & (df['senti'] == 1)] #delivery
df0_2 = df.loc[(df['label']==0) & (df['senti'] == 2)] #delivery
df1_1 = df.loc[(df['label']==1) & (df['senti'] == 1)] #size
df1_2 = df.loc[(df['label']==1) & (df['senti'] == 2)] #size
df2_1 = df.loc[(df['label']==2) & (df['senti'] == 1)] #color
df2_2 = df.loc[(df['label']==2) &  (df['senti'] == 2)] #color
df3_1 = df.loc[(df['label']==3) & (df['senti'] == 1)] #quality
df3_2 = df.loc[(df['label']==3) &  (df['senti'] == 2)] #quality

# 2. 데이터 전처리

In [57]:
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s

In [58]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_delivery_pos.txt', 'w')
for idx, item in df0_1.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [59]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_delivery_neg.txt', 'w')
for idx, item in df0_2.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [60]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_size_pos.txt', 'w')
for idx, item in df1_1.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [61]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_size_neg.txt', 'w')
for idx, item in df1_2.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [62]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_color_pos.txt', 'w')
for idx, item in df2_1.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [63]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_color_neg.txt', 'w')
for idx, item in df2_2.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [64]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_quality_pos.txt', 'w')
for idx, item in df3_1.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

In [96]:
text_data = open('/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_quality_neg.txt', 'w')
for idx, item in df3_2.iterrows():
  article = cleaning(item.sentence)
  text_data.write(article)
text_data.close()

# 3. 데이터 학습

In [66]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [67]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [84]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_delivery_pos.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_delivery_pos'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [87]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_delivery_neg.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_delivery_neg'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss


In [89]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_size_pos.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_size_pos'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss
500,3.501100


In [91]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_size_neg.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_size_neg'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.583700


In [92]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_color_pos.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_color_pos'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [93]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_color_neg.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_color_neg'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss


In [94]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_quality_pos.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_quality_pos'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss
500,3.769600
1000,3.540500


In [97]:
# you need to set parameters
train_file_path = "/content/drive/MyDrive/TM/Data/GPT/TXT/Articles_quality_neg.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/TM/Data/GPT/MODEL/Articles_quality_neg'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 1
save_steps = 500

# It takes about 30 minutes to train in colab.
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.781100


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
import re

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length, path):
    model_path = path
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return str(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

def extract_sentences(text):
    sentences = re.findall(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return sentences


In [ ]:
'''print("배송, 사이즈, 색상, 퀄리티 중 선택하시오 : ", end='')
path = input() #fine_tuned_gpt2_fahsion_delivery
sequence = input() # oil price
max_len = 50 #int(input()) # 20
if path == "배송":
    path = "/fine_tuned_gpt2_fashion_delivery"
elif path == "사이즈":
    path = "/fine_tuned_gpt2_fashion_size"
elif path == "색상":
    path = "/fine_tuned_gpt2_fashion_color"
else:
    path = "/fine_tuned_gpt2_fashion_quality"
res = generate_text(sequence, max_len, path)
print(res)
print(res.split(sep = '.')[1]+".")

배송, 사이즈, 색상, 퀄리티 중 선택하시오 : 배송
 
    - you got to choose. The price was excellent, I ordered a bottle which I had with me when it came out for sale, but I bought the bottle in the event this arrived.The package was super large, the delivery
 The price was excellent, I ordered a bottle which I had with me when it came out for sale, but I bought the bottle in the event this arrived.


In [ ]:
'''sequence = "Good"
max_len = 50
path = "/fine_tuned_gpt2_fashion_delivery"
temp = []
for i in range(300):
    res = generate_text(sequence, max_len, path)
    res = res.split(sep = '.')[1]+"."
    print(res)
    temp.append(res)